# Object Oriented Programming in ROS

## Sources
1. https://roboticsbackend.com/oop-with-ros-in-python/
2. http://wiki.ros.org/Services
3. http://wiki.ros.org/Topics



## Important Terms:-
1. ROS Topics
    1. A Publisher-Subscriber model
    2. Intended for unidirectional, streaming communication
    3. Eg, Sending the location of a robot
2. ROS Service
    1. A Client-Server model
    2. One time communication having a Request/Response (Unlike ROS Topics)
    3. Eg, Spawning a robot to simulator

## This Notebook's Application 
A simple number counter with:-
1. A ROS subscriber that receives a number from an external output. Upon reception, the number will be added to a counter.
2. A ROS publisher that publishes the new counter as soon as a number has been received and added to the existing counter.
3. A ROS service is used to reset the counter. If you call this service, the counter value will come back to 0.


## The Publisher

In [ ]:
#!/usr/bin/env python
# license removed for brevity
import rospy
from std_msgs.msg import Int64

def talker():
    pub = rospy.Publisher('/number', Int64, queue_size=10)
    rospy.init_node('talker', anonymous=True)
    rate = rospy.Rate(1) # 1hz
    counter_message = Int64()
    counter_message.data = 0
    while not rospy.is_shutdown():
        counter_message.data += 1
        pub.publish(counter_message)
        rate.sleep()

if __name__ == '__main__':
    try:
        talker()
    except rospy.ROSInterruptException:
        pass

## Subscriber without OOPs

In [ ]:
#!/usr/bin/env python

import rospy
from std_msgs.msg import Int64
from std_srvs.srv import SetBool

counter = 0
pub = None

def callback_number(msg):
    global counter
    counter += msg.data
    new_msg = Int64()
    new_msg.data = counter
    rospy.loginfo("Received Data: {}, Counter Value: {}".format(msg.data, new_msg.data))
    pub.publish(new_msg)

def callback_reset_counter(req):
    if req.data:
        global counter
        counter = 0
        return True, "Counter has been successfully reset"
    return False, "Counter has not been reset"

if __name__ == '__main__':
    rospy.init_node('number_counter') 
    pub = rospy.Publisher("/number_count", Int64, queue_size=10)
    sub = rospy.Subscriber("/number", Int64, callback_number)
    reset_service = rospy.Service("/reset_counter", SetBool, callback_reset_counter)
    rospy.spin()

### Drawbacks 
1. Unscalable - Global Declaration of variables and publishers makes it difficult to scale the application upwards.
2. Unpredictable - The behaviour is difficult to predict when the same global variable is used by multiple entities

## Subscriber with OOPs

In [ ]:
#!/usr/bin/env python

import rospy
from std_msgs.msg import Int64
from std_srvs.srv import SetBool

class NumberCounter:

    def __init__(self):
        self.counter = 0
        self.pub = rospy.Publisher("/number_count", Int64, queue_size=10)
        self.number_subscriber = rospy.Subscriber("/number", Int64, self.callback_number)
        self.reset_service = rospy.Service("/reset_counter", SetBool, self.callback_reset_counter)

    def callback_number(self, msg):
        self.counter += msg.data
        new_msg = Int64()
        new_msg.data = self.counter
        rospy.loginfo("Received Data: {}, Counter Value: {}".format(msg.data, new_msg.data))
        self.pub.publish(new_msg)

    def callback_reset_counter(self, req):
        if req.data:
            self.counter = 0
            return True, "Counter has been successfully reset"
        return False, "Counter has not been reset"

if __name__ == '__main__':
    rospy.init_node('number_counter')
    NumberCounter()
    rospy.spin()

## Comparison

1. Scalable - Can create multiple instances of the same class
2. Encapsulation - Each NumberCounter Object can access itself
2. Modular - Different Functionalities can be written without dirupting the old

In [ ]:
# Publisher